In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.utils import set_random_seed as set_seed
set_seed(777)


In [2]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('Wind_deseason_15.csv', parse_dates=[0], index_col=0)


In [3]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 
        
            

In [4]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['309_WIND_1'].shift(i+1)
    
    return df_new

In [5]:

def model1Train(X,Y):
     
    mod = Sequential()
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1)))
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(x = X, y = Y,epochs=20,verbose = 1)#keep low for now, edit later
    
    return mod
    
    
    

In [6]:
def model2Train(gen):
     
    mod = Sequential()
    mod.add(LSTM(1, activation='sigmoid', input_shape=(107, 1)))
   
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(gen,epochs=5,verbose = 1)#keep low for now, edit later
    
    return mod
    

In [7]:
def TestModel(X,test, mod, scaler):
    
    MSEs = []
    
    for i in range(len(X)):
        array = np.array(X[i])
        
        pred = mod.predict(array.reshape(1,107,1))
        print(pred)
        #pred = scaler.inverse_transform(pred)
        
        unscaled_pred = scaler.inverse_transform(pred)
        unscaled_test = scaler.inverse_transform([[test[i]]])
       
        
        MSEs.append(mse(unscaled_test[0],unscaled_pred[0]))
        
        
    return MSEs

In [8]:
#def modelNaiveTest(df):

    
    
    


In [9]:
def runModels(df, last):
    
    scaler = MinMaxScaler()
    
    bound = TrainTest(df)
    
    train = df[0:bound]
    test = df[bound:]
    
    
    scaler.fit(train['309_WIND_1'].to_frame())
    
    scaled_df = scaler.transform(df['309_WIND_1'].to_frame())
    scaled_last = scaler.transform(last['309_WIND_1'].to_frame())
    scaled_df = np.append(scaled_last, scaled_df)
    
    
    generator = TimeseriesGenerator(scaled_df[107:bound+107], scaled_df[107:bound+107], length=107, batch_size=1)
                                      
    

    scaled_df = pd.DataFrame(scaled_df, columns = ['309_WIND_1'])
    
    lag = lags(scaled_df, 107)
    
    lag = lag[lag.columns[::-1]]
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    
    
    X_test = lagged_test.drop('309_WIND_1', axis = 1).values.tolist()
    Y_test = lagged_test['309_WIND_1'].values.tolist()
  
    
    model2 = model2Train(generator)
    
    Tests = TestModel(X_test, Y_test, model2, scaler)
    
    
    print(Tests)
    
   
    
    return Tests#, mse2

    

In [10]:


df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]
print(len(df1))

df2 = df[df.index.month.isin([4,5,6])]

df3 = df[df.index.month.isin([7,8,9])]

df4 = df[df.index.month.isin([10,11,12])]

8735


In [39]:
# mse1 = runModels(df1,df4.iloc[-107:])


In [40]:
#mse3 = runModels(df2,df1.iloc[-107:])

In [ ]:
mse5 = runModels(df3, df2.iloc[-107:])

C:\Users\bvits\AppData\Local\Temp\ipykernel_15496\1882170497.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['309_WIND_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_15496\1882170497.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['309_WIND_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_15496\1882170497.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1)                 12        
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
6958/6958 [==============================] - 202s 29ms/step - loss: 0.0319 - accuracy: 1.4372e-04
Epoch 2/5
6958/6958 [==============================] - 176s 25ms/step - loss: 0.0056 - accuracy: 2.8744e-04
Epoch 3/5
6958/6958 [==============================] - 194s 28ms/step - loss: 0.0023 - accuracy: 2.8744e-04
Epoch 4/5
6958/6958 [==============================] - 219s 31ms/step - loss: 0.0018 - accuracy: 2.8744e-04
Epoch 5/5
1/1 [====

1/1 [==============================] - 0s 23ms/step
[[0.13567293]]
1/1 [==============================] - 0s 20ms/step
[[0.13335162]]
1/1 [==============================] - 0s 20ms/step
[[0.13104606]]
1/1 [==============================] - 0s 22ms/step
[[0.13096291]]
1/1 [==============================] - 0s 23ms/step
[[0.13041246]]
1/1 [==============================] - 0s 23ms/step
[[0.12935132]]
1/1 [==============================] - 0s 20ms/step
[[0.1291995]]
1/1 [==============================] - 0s 17ms/step
[[0.12941271]]
1/1 [==============================] - 0s 23ms/step
[[0.12972033]]
1/1 [==============================] - 0s 20ms/step
[[0.12979203]]
1/1 [==============================] - 0s 24ms/step
[[0.13045382]]
1/1 [==============================] - 0s 20ms/step
[[0.12996614]]
1/1 [==============================] - 0s 21ms/step
[[0.12944198]]
1/1 [==============================] - 0s 24ms/step
[[0.13216501]]
1/1 [==============================] - 0s 22ms/step
[[0.1299947

1/1 [==============================] - 0s 21ms/step
[[0.08182096]]
1/1 [==============================] - 0s 20ms/step
[[0.08009028]]
1/1 [==============================] - 0s 21ms/step
[[0.07929468]]
1/1 [==============================] - 0s 35ms/step
[[0.07848251]]
1/1 [==============================] - 0s 21ms/step
[[0.07827806]]
1/1 [==============================] - 0s 21ms/step
[[0.076756]]
1/1 [==============================] - 0s 19ms/step
[[0.08331275]]
1/1 [==============================] - 0s 21ms/step
[[0.08602035]]
1/1 [==============================] - 0s 46ms/step
[[0.08692467]]
1/1 [==============================] - 0s 42ms/step
[[0.08578002]]
1/1 [==============================] - 0s 21ms/step
[[0.08697462]]
1/1 [==============================] - 0s 24ms/step
[[0.08862269]]
1/1 [==============================] - 0s 36ms/step
[[0.08605099]]
1/1 [==============================] - 0s 19ms/step
[[0.08516622]]
1/1 [==============================] - 0s 21ms/step
[[0.08617651

1/1 [==============================] - 0s 22ms/step
[[0.04241514]]
1/1 [==============================] - 0s 19ms/step
[[0.04239857]]
1/1 [==============================] - 0s 19ms/step
[[0.04180896]]
1/1 [==============================] - 0s 18ms/step
[[0.04201376]]
1/1 [==============================] - 0s 25ms/step
[[0.04298067]]
1/1 [==============================] - 0s 23ms/step
[[0.04296446]]
1/1 [==============================] - 0s 22ms/step
[[0.04283798]]
1/1 [==============================] - 0s 23ms/step
[[0.043962]]
1/1 [==============================] - 0s 24ms/step
[[0.042822]]
1/1 [==============================] - 0s 20ms/step
[[0.04278445]]
1/1 [==============================] - 0s 26ms/step
[[0.044626]]
1/1 [==============================] - 0s 22ms/step
[[0.04318595]]
1/1 [==============================] - 0s 24ms/step
[[0.04412389]]
1/1 [==============================] - 0s 23ms/step
[[0.04444945]]
1/1 [==============================] - 0s 20ms/step
[[0.04335654]]
1

...

1/1 [==============================] - 0s 26ms/step
[[0.19315189]]
1/1 [==============================] - 0s 26ms/step
[[0.17625618]]
1/1 [==============================] - 0s 27ms/step
[[0.1551497]]
1/1 [==============================] - 0s 28ms/step
[[0.159033]]
1/1 [==============================] - 0s 27ms/step
[[0.14620322]]
1/1 [==============================] - 0s 22ms/step
[[0.16173482]]
1/1 [==============================] - 0s 23ms/step
[[0.16830838]]
1/1 [==============================] - 0s 22ms/step
[[0.15394688]]
1/1 [==============================] - 0s 22ms/step
[[0.16972047]]
1/1 [==============================] - 0s 24ms/step
[[0.16530323]]
1/1 [==============================] - 0s 31ms/step
[[0.16600192]]
1/1 [==============================] - 0s 26ms/step
[[0.13783687]]
1/1 [==============================] - 0s 27ms/step
[[0.11668557]]
1/1 [==============================] - 0s 19ms/step
[[0.11886281]]
1/1 [==============================] - 0s 22ms/step
[[0.09493458]

1/1 [==============================] - 0s 22ms/step
[[0.11509943]]
1/1 [==============================] - 0s 20ms/step
[[0.11471719]]
1/1 [==============================] - 0s 23ms/step
[[0.1138469]]
1/1 [==============================] - 0s 24ms/step
[[0.11630899]]
1/1 [==============================] - 0s 22ms/step
[[0.11564833]]
1/1 [==============================] - 0s 21ms/step
[[0.11368316]]
1/1 [==============================] - 0s 23ms/step
[[0.11535347]]
1/1 [==============================] - 0s 19ms/step
[[0.11449236]]
1/1 [==============================] - 0s 19ms/step
[[0.11439598]]
1/1 [==============================] - 0s 20ms/step
[[0.11487782]]
1/1 [==============================] - 0s 21ms/step
[[0.11384326]]
1/1 [==============================] - 0s 23ms/step
[[0.11408645]]
1/1 [==============================] - 0s 21ms/step
[[0.11516649]]
1/1 [==============================] - 0s 22ms/step
[[0.11492342]]
1/1 [==============================] - 0s 22ms/step
[[0.1151121

1/1 [==============================] - 0s 25ms/step
[[0.08380318]]
1/1 [==============================] - 0s 28ms/step
[[0.08218694]]
1/1 [==============================] - 0s 33ms/step
[[0.08178759]]
1/1 [==============================] - 0s 25ms/step
[[0.08402193]]
1/1 [==============================] - 0s 24ms/step
[[0.08641493]]
1/1 [==============================] - 0s 24ms/step
[[0.08563817]]
1/1 [==============================] - 0s 23ms/step
[[0.08420944]]
1/1 [==============================] - 0s 22ms/step
[[0.08127308]]
1/1 [==============================] - 0s 23ms/step
[[0.0812887]]
1/1 [==============================] - 0s 23ms/step
[[0.08263862]]
1/1 [==============================] - 0s 26ms/step
[[0.08234847]]
1/1 [==============================] - 0s 26ms/step
[[0.08264124]]
1/1 [==============================] - 0s 21ms/step
[[0.08197021]]
1/1 [==============================] - 0s 21ms/step
[[0.07910454]]
1/1 [==============================] - 0s 48ms/step
[[0.0808523

1/1 [==============================] - 0s 19ms/step
[[0.08965397]]
1/1 [==============================] - 0s 21ms/step
[[0.08724713]]
1/1 [==============================] - 0s 35ms/step
[[0.08255792]]
1/1 [==============================] - 0s 46ms/step
[[0.08055007]]
1/1 [==============================] - 0s 59ms/step
[[0.07993364]]
1/1 [==============================] - 0s 51ms/step
[[0.07916653]]
1/1 [==============================] - 0s 20ms/step
[[0.07970941]]
1/1 [==============================] - 0s 21ms/step
[[0.08008146]]
1/1 [==============================] - 0s 31ms/step
[[0.07882011]]
1/1 [==============================] - 0s 41ms/step
[[0.07997739]]
1/1 [==============================] - 0s 64ms/step
[[0.07805121]]
1/1 [==============================] - 0s 21ms/step
[[0.07806754]]
1/1 [==============================] - 0s 49ms/step
[[0.08026922]]
1/1 [==============================] - 0s 52ms/step
[[0.08030975]]
1/1 [==============================] - 0s 47ms/step
[[0.080314

1/1 [==============================] - 0s 21ms/step
[[0.14858699]]
1/1 [==============================] - 0s 19ms/step
[[0.15218705]]
1/1 [==============================] - 0s 29ms/step
[[0.15928084]]
1/1 [==============================] - 0s 23ms/step
[[0.141348]]
1/1 [==============================] - 0s 20ms/step
[[0.13902521]]
1/1 [==============================] - ETA: 0s

In [42]:
#mse7 = runModels(df4, df3.iloc[-107:])

In [43]:
#print('season 1')

#print('model 1:')
#print(mse1)
#print('model Naive:')
#print(mse2)


In [44]:
#print('season 1')

#print('model 1:')
#print(mse3)
#print('model Naive:')
#print(mse4)


In [45]:
#print('season 1')

#print('model 1:')
#print(mse5)
#print('model Naive:')
#print(mse6)


In [46]:
#print('season 1')

#print('model 1:')
#print(mse7)
#print('model Naive:')
#print(mse8)


In [47]:
#'Season 1':mse2,'Season 2':mse4,'Season 4':mse8
d = {'Season 3':mse5}
#df_errors_Naive = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
#d = {'Season1':mse1,'Season2':mse3,'Season3':mse5,'Season4':mse7}
df_errors_LSTM = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
d#f_errors_Naive.to_csv('Naive_errors.csv')
df_errors_LSTM.to_csv('LSTM_errors_not_scaled_s3.csv')

